# Extracción de los intereses de una cuenta de Twitter hasta 2º grado
Este código se desarrolló para descargar los datos de las cuentas que sigue Vox y las cuentas que siguen las cuentas seguidas por Vox. De esta manera se obtendrian los intereses de la cuenta de Vox en Twitter hasta segundo grado. El mismo código es aplicable para cualquier otra cuenta para obtener un grafo y asi analizar y conocer estos intereses de las cuentas que sigue la principal.

In [5]:
# Ponemos los identificadores (se borran por motivos de privacidad)
consumer_key = 'XXXX'
consumer_secret = 'XXXX'
access_token = 'XXXX'
access_token_secret = 'XXXX'

In [6]:
# Importamos 
import tweepy
import json
import pandas as pd
import time
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [7]:
# Preparamos la autenticación
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Preparamos el módulo api de Tweepy y le ponemos que espere cuando superemos los limites permitidos
api = tweepy.API(auth, wait_on_rate_limit=True)


La intencion es descargar los datos de las cuentas que sigue Vox en Twitter, y luego descargar los datos de las cuentas que siguen los seguidos por Vox. En definitiva, se pretende conseguir las cuentas hasta segundo grado de los seguidos por Vox.
Para ello primero descargamos una lista con los ids de las cuentas que sigue Vox para luego usar esa lista para descargar todos los datos de cada cuenta. Luego usar estos seguidos por Vox para realizar el mismo proceso para cada una de las cuentas seguidas por Vox.

In [9]:
# Creamos el nombre del cual vamos a extraer los ids de las cuentas a las que sigue
screen_name="vox_es"

# Creamos una ista vacia donde guardaremos los ids
user_id_list = []

# Hacemos un bulce que extraerá y guardará los ids de las cuentas
for page in tweepy.Cursor(api.friends_ids, screen_name).pages():
    user_id_list.extend(page)
    # Para cada vuelta del bucle lo paramos 60 segundos por los limites de Twitter
    time.sleep(60)

In [91]:
# Parte del código lo hemos encontrado en 
#https://stackoverflow.com/questions/43782889/tweepy-how-can-i-look-up-more-than-100-user-screen-names

# Creamos una funcion que extrae la informacion de las cuentas de los ids anteriores
def lookup_user_list(user_id_list, api):
    # Creamos una lista vacia donde guardaremos todos los datos
    full_users = []
    # Creamos una variable para el futuro bucle con los elementos que tenga la lista
    users_count = len(user_id_list) 
    # Vamos a hacer la extracción de 100 en 100 cuentas ya que es el limite de Twitter en estos casos
    try:
        for i in range((users_count / 100) + 1):
            x = i*100
            y = min((i+1)*100, users_count)
            # Ponemos la condición para que si x e y son diferentes haga la extracción sino pasará al siguiente y terminará
            if x != y:
                full_users.extend(api.lookup_users(user_ids=user_id_list[i*100:min((i+1)*100, users_count)]))
        return full_users
    except tweepy.TweepError:
        print 'Something went wrong, quitting...'

# Ejecutamos la funcion y guardamos el resutlado
results = lookup_user_list(user_id_list, api)

In [92]:
# establecemos la estructura de los datos que vamos a guardar de cada usuario
all_users = [{'id': user.id,
             'Name': user.name,
             'Statuses Count': user.statuses_count,
             'Friends Count': user.friends_count,
             'Screen Name': user.screen_name,
             'Followers Count': user.followers_count,
             'Location': user.location,
             'Language': user.lang,
             'Created at': user.created_at,
             'Time zone': user.time_zone,
             'Geo enable': user.geo_enabled,
             'Description': user.description,
             'Seguido por': screen_name,
             'Protegido': user.protected}
             for user in results]

# Pasamos los datos a un dataframe
df = pd.DataFrame(all_users)

In [13]:
# Lo guardamos en un csv por si acaso
df.to_csv('1_Seguidos_Vox.csv', index=False, encoding='utf-8')

Con esto ya tenemos los datos de todas las cuentas que sigue Vox a dia 17/12/2020, ahora tenemos que descargar la de los que siguen los seguidos por vox.

In [14]:
# Vemos la cabecera de los datos
df.head(5)

,Created at,Description,Followers Count,Friends Count,Geo enable,Language,Location,Name,Protegido,Screen Name,Seguido por,Statuses Count,Time zone,id
0,2020-11-03 07:43:19,Yo también creo que Jack cabía en el tablón de...,10222,167,False,None,,Alonso 🇪🇸,False,AlonsoDm2,vox_es,1189,None,1323531147104686080
1,2009-04-26 17:20:37,TAC is a bimonthly print and daily digital mag...,53637,447,False,None,"Washington, DC",The American Conservative,False,amconmag,vox_es,132276,None,35511525
2,2019-09-16 12:04:51,por que se llama asno si tiene una foto de un ...,8912,244,False,None,Society,asnito 🥀,False,nootnootasno,vox_es,543,None,1173568390927147008
3,2020-04-08 20:42:29,"Analfabeto, Cansado, Socialdemócrata, Nyno Fan...",1054,1510,False,None,,👑 King Freedrich 🇪🇸,False,KingFriedrich4,vox_es,6941,None,1247988151873867776
4,2011-06-08 11:44:10,"Yo a las nueve de la mañana soy comunista, a l...",510,1113,False,None,En algún lugar...,Teniente Damm,False,Tte_Damm,vox_es,18040,None,313262677


Si revisamos un poco estos datos vemos que tenemos que filtrar ya que no podemos descargar todos los seguidos de todas las cuentas que sigue Vox ya que tendriamos millones de datos que desenfocarian la intención de la descarga.

In [15]:
# Vemos que hay usuarios que siguen a un millon de cuentas
max(df['Friends Count'])

1064009L

Vemos la cuenta que tiene más seguidos. Si nos fijamos es una cuenta que sigue a mas de 1 millon de cuentas y es seguida por otro millon. En este caso es bastante obvio que es una cuenta de una persona que no le importa lo que digan las personas que sigue, es una persona que no sigue a otros por lo que pongan o porque tengan interes en su cuenta sino que es una persona que suele seguir a los que le siguen. Por lo tanto este tipo de cuentas no nos interesan. Lo que queremos conocer son las cuentas que le interesan a Vox que son las que ya tenemos descargadas, y ahora queremos saber que cuentas les interesan a las cuentas que siguen Vox. Por esta razon, vamos a filtrar la base poniendo como limite las cuentas que siguen a un máximo de 1000 personas por dar cierto margen. Es bastante obvio que una persona que sigue a 500 cuentas no llegará a ver los tweets de todas las cuentas pero si puede reflejar que son cuentas que le interesan, pero una persona que sigue más de 1000 lo más probable es que las siga porque les siguen o para conseguir seguidores, no porque les interesen esas cuentas. Ademas tenemos que tener en cuenta que hay cuentas protegidas a las que no podemos acceder a sus seguidos por lo que tambien las eliminaremos para la sigueinte busqueda.

In [16]:
# Vemos el caso anterior con tantos seguidores para confirmar nuestra teoria de que sigue mucho y le siguen muchos
df[df['Friends Count'] == 1064009 ]

,Created at,Description,Followers Count,Friends Count,Geo enable,Language,Location,Name,Protegido,Screen Name,Seguido por,Statuses Count,Time zone,id
817,2009-06-22 15:20:41,Presidente Editor Diario El Nacional,1805035,1064009,True,None,"El Nacional, Los Cortijos",Miguel H Otero,False,miguelhotero,vox_es,28179,None,49658852


In [21]:
print("Cuentas que siguen más de 1000 cuentas: " + str(len( df[df['Friends Count'] > 1000 ])))

Cuentas que siguen más de 1000 cuentas: 534


In [22]:
print("Cuentas protegidas: " + str(len( df[df['Protegido'] == True ])))

Cuentas protegidas: 16


Ahora vamos a guardar en otro df las cuentas que sigue vox ya que tenemos que eliminar la cuentas que sigan mas de 1000 cuentas y las que esten protegidas para que no nos de error el proceso. Guardamos las dos bases ya que la base parcial que vamos a usar ahora solo la usaremos para la descarga de datos, cuando ya tengamos todos los datos tendremos que borrar la parte de la base parcial de Vox y añadir la completa con todos los seguidos.

In [28]:
# Duplicamos la base para luego filtrarla
df_seguidosVox = df
df_seguidosVox.head(5)

,Created at,Description,Followers Count,Friends Count,Geo enable,Language,Location,Name,Protegido,Screen Name,Seguido por,Statuses Count,Time zone,id
0,2020-11-03 07:43:19,Yo también creo que Jack cabía en el tablón de...,10222,167,False,None,,Alonso 🇪🇸,False,AlonsoDm2,vox_es,1189,None,1323531147104686080
1,2009-04-26 17:20:37,TAC is a bimonthly print and daily digital mag...,53637,447,False,None,"Washington, DC",The American Conservative,False,amconmag,vox_es,132276,None,35511525
2,2019-09-16 12:04:51,por que se llama asno si tiene una foto de un ...,8912,244,False,None,Society,asnito 🥀,False,nootnootasno,vox_es,543,None,1173568390927147008
5,2020-08-08 08:16:08,Espero que poniendo velas y peluches acojonemo...,13757,333,False,None,Fuente Alamo de Murcia,Un murciano encabronao,False,MurcianoUn,vox_es,3032,None,1292011665148518407
6,2017-01-11 18:17:38,"Si me tenéis en comunio, vended.",135847,845,True,None,ESPAÑA,Kilgore,False,billkilgore_,vox_es,11788,None,819246908191494144


In [29]:
# Eliminamos las cuentas que tienen mas de 1000 seguidos
df = df.drop(df[df['Friends Count'] > 1000].index)
df.head(5)

,Created at,Description,Followers Count,Friends Count,Geo enable,Language,Location,Name,Protegido,Screen Name,Seguido por,Statuses Count,Time zone,id
0,2020-11-03 07:43:19,Yo también creo que Jack cabía en el tablón de...,10222,167,False,None,,Alonso 🇪🇸,False,AlonsoDm2,vox_es,1189,None,1323531147104686080
1,2009-04-26 17:20:37,TAC is a bimonthly print and daily digital mag...,53637,447,False,None,"Washington, DC",The American Conservative,False,amconmag,vox_es,132276,None,35511525
2,2019-09-16 12:04:51,por que se llama asno si tiene una foto de un ...,8912,244,False,None,Society,asnito 🥀,False,nootnootasno,vox_es,543,None,1173568390927147008
5,2020-08-08 08:16:08,Espero que poniendo velas y peluches acojonemo...,13757,333,False,None,Fuente Alamo de Murcia,Un murciano encabronao,False,MurcianoUn,vox_es,3032,None,1292011665148518407
6,2017-01-11 18:17:38,"Si me tenéis en comunio, vended.",135847,845,True,None,ESPAÑA,Kilgore,False,billkilgore_,vox_es,11788,None,819246908191494144


In [31]:
# Eliminamos las cuentas que son protegidas
df = df.drop(df[df['Protegido'] == True].index)
df.head(5)

,Created at,Description,Followers Count,Friends Count,Geo enable,Language,Location,Name,Protegido,Screen Name,Seguido por,Statuses Count,Time zone,id
0,2020-11-03 07:43:19,Yo también creo que Jack cabía en el tablón de...,10222,167,False,None,,Alonso 🇪🇸,False,AlonsoDm2,vox_es,1189,None,1323531147104686080
1,2009-04-26 17:20:37,TAC is a bimonthly print and daily digital mag...,53637,447,False,None,"Washington, DC",The American Conservative,False,amconmag,vox_es,132276,None,35511525
2,2019-09-16 12:04:51,por que se llama asno si tiene una foto de un ...,8912,244,False,None,Society,asnito 🥀,False,nootnootasno,vox_es,543,None,1173568390927147008
5,2020-08-08 08:16:08,Espero que poniendo velas y peluches acojonemo...,13757,333,False,None,Fuente Alamo de Murcia,Un murciano encabronao,False,MurcianoUn,vox_es,3032,None,1292011665148518407
6,2017-01-11 18:17:38,"Si me tenéis en comunio, vended.",135847,845,True,None,ESPAÑA,Kilgore,False,billkilgore_,vox_es,11788,None,819246908191494144


In [27]:
print("finalmente tenemos las siguientes cuentas: " +str(len( df)))

finalmente tenemos las siguientes cuentas: 685


Ahora ya tenemos las cuentas de las que vamos a extraer las cuentas que siguen, y repetimos el mismo proceso que con Vox pero haciendo un bucle por cada seguido de Vox.

In [10]:
# Hacemos una lista con los nombres de las 685 cuentas seguidas por Vox con menos de 1000 seguidores y publicas
lista_seguidores_Vox = list(df['Screen Name'])

# Iteramos el proceso que hemos hecho con Vox por cada cuenta de la lista anterior
for i in range(len(lista_seguidores_Vox)):
    screen_name = lista_seguidores_Vox[i]
    
    user_id_list = []
    for page in tweepy.Cursor(api.friends_ids, screen_name).pages():
        user_id_list.extend(page)
        time.sleep(60)
    
    results = lookup_user_list(user_id_list, api)

    all_users = [{'id': user.id,
                 'Name': user.name,
                 'Statuses Count': user.statuses_count,
                 'Friends Count': user.friends_count,
                 'Screen Name': user.screen_name,
                 'Followers Count': user.followers_count,
                 'Location': user.location,
                 'Language': user.lang,
                 'Created at': user.created_at,
                 'Time zone': user.time_zone,
                 'Geo enable': user.geo_enabled,
                 'Description': user.description,
                 'Seguido por': screen_name,}
                 for user in results]

    df2 = pd.DataFrame(all_users)    
    
    df = pd.concat([df, df2])


In [83]:
# Eliminamos de la base las cuentas que hemos filtrado de los seguidores de Vox para luego añadir la lista completa
df = df.drop(df[df['Seguido por'] == vox_es].index)
df = pd.concat([df, df_seguidosVox])


In [95]:
# Guardamso la base definitiva
df.to_csv('3_Red_Vox_2Grado.csv', index=False, encoding='utf-8')

Ahora que ya tenemos los datos podemos pasar a crear el grafo

In [15]:
# Cada fila del datset será una arista para el grafo ya que representan una cuenta que sigue a otra
print("Aristas en total: " + str(len(df)))

Aristas en total: 316581


In [18]:
# Creamos una lista con los usuarios de las dos columnas para tener todas las cuentas
nodos1 = list(df['Screen Name'])
nodos2 = list(df['Seguido por'])

# Juntamos las dos listas
nodos1 += nodos2

# Como hay cuentas que se repetiran, nos quedamos con una lista donde solo tengamos una vez cada cuenta
lista_nodos = set(nodos1)

print("Nodos en total: " + str(len(lista_nodos)))

Nodos en total: 127526


Ahora vamos a crear el grafo, el cual tiene que ser dirigido ya que las relaciones no son correspondidas y nuestro interes va solo en una dirección, a quién sigue?

In [25]:
# Creamos un grafo dirigido
G = nx.DiGraph()

# Añadimos los nodos usando sacando los unicos de los dos puntos de las conexiones
G.add_nodes_from(lista_nodos)
# Hacemos un bucle para asignar los enlaces y sus pesos
G.add_edges_from(zip(df['Seguido por'], df['Screen Name']))


In [26]:
# Guardamos el Grafo en bruto para luego pasarlo a Gephi
nx.write_graphml(G, "Red_Vox.graphml")